# OSP model



### Import libraries

In [ ]:
%load_ext lab_black
import meta, evaluate, h5py, data_wrangling, modeling
import numpy as np
import matplotlib.pyplot as plt

### Load model configuration

In [ ]:
# Get 0, 2, 4 noise sample model
models = ["0098", "0518", "0938"]

cfg = meta.model_cfg(f"models/O2P_replication_set12_r{models[2]}/model_config.json")

### Evaluate routine... 

In [ ]:
data = data_wrangling.my_data(cfg)
from tensorflow.keras import Model
from tensorflow.keras.layers import Layer, Input, concatenate, multiply, RepeatVector
from tensorflow.keras.optimizers import Adam, SGD


def build_model(training=True):
    """
    Create Keras model
    Note that:
    For structural things, such as repeat vector, should build within the model
    For Static calculation of input, it is easier to modify, should build within sample generator
    """

    cfg.noise_on() if training is True else cfg.noise_off()
    input_o = Input(shape=(cfg.input_dim,), name="Input_O")
    input_o_t = RepeatVector(cfg.n_timesteps, name="Input_Ot")(input_o)

    if cfg.use_semantic == True:
        raw_s_t = Input(shape=(cfg.n_timesteps, cfg.output_dim), name="Plaut_St")
        input_p = Input(shape=(cfg.output_dim,), name="input_P")
        input_p_t = RepeatVector(cfg.n_timesteps, name="Teaching_Pt")(input_p)
        input_s_t = multiply([raw_s_t, input_p_t], name="Input_St")
        combined = concatenate([input_o_t, input_s_t], name="Combined_input")
        rnn_model = rnn(cfg)(combined)
        model = Model([input_o, raw_s_t, input_p], rnn_model)

    else:
        rnn_model = modeling.rnn(cfg)(input_o_t)
        model = Model(input_o, rnn_model)

    if cfg.optimizer == "adam":
        op = Adam(
            learning_rate=cfg.learning_rate, beta_1=0.9, beta_2=0.999, amsgrad=False
        )

    elif cfg.optimizer == "sgd":
        op = SGD(cfg.learning_rate)

    model.compile(
        loss=modeling.zer_bce, optimizer=op, metrics=["BinaryAccuracy", "mse"]
    )

    model.summary()
    return model


model = build_model(training=False)

In [ ]:
# Strain full model

strain = evaluate.strain_eval(cfg, data, model)
strain.start_evaluate(
    test_use_semantic=False,
    output=cfg.path_model_folder + "result_strain_item_with_slot_SSE.csv",
)

# # Grain
# grain = evaluate.grain_eval(cfg, data, model)
# grain.start_evaluate(output=cfg.path_model_folder + "result_grain_item.csv")

# # Taraban
# taraban = evaluate.taraban_eval(cfg, data, model)
# taraban.start_evaluate(
#     test_use_semantic=False, output=cfg.path_model_folder + "result_taraban_item.csv"
# )

# # Glushko
# glushko = evaluate.glushko_eval(cfg, data, model)
# glushko.start_evaluate(False, cfg.path_model_folder + "result_glushko_item.csv")

### Training history

In [ ]:
vis = evaluate.vis(cfg.path_model_folder)
vis.training_hist()

### Strain plots

In [ ]:
vis.plot_dev_interactive("acc", ["strain", "grain"]).properties(
    title="Accuracy in Strain and Grain"
)

### Grain plots

In [ ]:
small = vis.plot_dev_interactive("acc_small_grain", exp=["grain"]).properties(
    title="Small Grain Response"
)
large = vis.plot_dev_interactive("acc_large_grain", exp=["grain"]).properties(
    title="Large Grain Response"
)

(small | large).properties(title="Accuracy of Grain by response and condition")

### Taraban plots

In [ ]:
vis.plot_dev_interactive("acc", ["taraban"]).properties(
    title="Accuracy in Taraban by condition"
)

### Glushko plots

In [ ]:
vis.plot_dev_interactive("acc", ["glushko"]).properties(
    title="Accuracy in Glushko by condition"
)

### Words vs. Nonwords

In [ ]:
wnw_zr = vis.plot_wnw(["INC_HF"], ["unambiguous"]).properties(
    title="Strain (INC_HF) vs. Grain (UN)"
)

all_taraban_conds = list(vis.cdf.loc[vis.cdf.exp == "taraban", "cond"].unique())
wnw_tg = vis.plot_wnw(all_taraban_conds, ["Exception", "Regular"]).properties(
    title="Taraban (all) vs. Glushko (NW)"
)
wnw_zr | wnw_tg

### Weights

In [ ]:
vis.weight.violinplot(cfg.path_plot_folder + "weight_violinplot.png")

In [ ]:
ax = vis.weight.df.boxplot(
    column="abs_weight", by="matrix", grid=False, figsize=(10, 5)
)

In [ ]:
vis.weight.boxplot(cfg.path_plot_folder + "weight_boxplot.png")

In [ ]:
vis.weight.basic_stat()

In [ ]:
!jupyter nbconvert --output-dir=$cfg.path_model_folder --to html OSP_master.ipynb